In [1]:
!pip install fastapi uvicorn nest_asyncio pyngrok opencv-python onnxruntime tensorflow
!pip install python-multipart
!ngrok config add-authtoken 2w632UmskfhkiCHNqk8koimwg08_89pfMfZsEGcJj2q56zHkD

Authtoken saved to configuration file: C:\Users\Fayroz\AppData\Local/ngrok/ngrok.yml


In [2]:
!pip install pycaret


In [ ]:
#this is the orginal FastAPI code that we enhanced in the next cell to allow the video spliting by analysing each chunk
#that will be sent by the RESTFUL API



# from fastapi import FastAPI, UploadFile, File
# from fastapi.responses import JSONResponse
# import shutil
# import pandas as pd
# import traceback
# from pycaret.classification import load_model
# from pose_feature_extractor import extract_features_from_video

# app = FastAPI()

# # Load PyCaret pipeline
# model = load_model("my_first_pipeline4")

# @app.post("/analyze_video/")
# async def analyze_video(file: UploadFile = File(...)):
#     try:
#         video_path = "temp_video.mp4"
#         with open(video_path, "wb") as buffer:
#             shutil.copyfileobj(file.file, buffer)

#         df = extract_features_from_video(video_path)
#         df.to_csv("player_features.csv", index=False)

#         prediction = model.predict(df)[0]

#         return {
#             "prediction": prediction,
#             "features": df.iloc[0].to_dict()
#         }

#     except Exception as e:
#         return JSONResponse(
#             status_code=500,
#             content={
#                 "error": str(e),
#                 "traceback": traceback.format_exc()
#             }
#         )

In [5]:
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
import shutil
import pandas as pd
import traceback
from pycaret.classification import load_model
from pose_feature_extractor import extract_features_from_video

app = FastAPI()

model = load_model("my_first_pipeline4")

@app.post("/process_chunk/")
async def process_chunk(file: UploadFile = File(...)):
    try:
        video_path = f"temp_chunks/{file.filename}"
        with open(video_path, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)

        df = extract_features_from_video(video_path)
        df.to_csv(f"features_csv/{file.filename}_features.csv", index=False)

        prediction = model.predict(df)[0]

        return {
            "filename": file.filename,
            "prediction": prediction,
            "features": df.iloc[0].to_dict()  
        }

    except Exception as e:
        return JSONResponse(
            status_code=500,
            content={
                "error": str(e),
                "traceback": traceback.format_exc()
            }
        )


Transformation Pipeline and Model Successfully Loaded


In [4]:
import tensorflow as tf
print(tf.__version__)


2.18.0


In [ ]:
import nest_asyncio
import uvicorn
from pyngrok import ngrok

nest_asyncio.apply()

# Open ngrok tunnel to port 8000
public_url = ngrok.connect(8000)
print("🚀 Public URL:", public_url)

# Run FastAPI app
uvicorn.run(app, host="0.0.0.0", port=8000)

#The public url changes everytime this code runs so it is not stable so each time we run it we take the 
# New URL For Example:https://8cf8-196-137-30-105.ngrok-free.app and put it in the env file in this form: https://8cf8-196-137-30-105.ngrok-free.app/process_chunk/

ERROR:pyngrok.process.ngrok:t=2025-05-25T09:48:18+0300 lvl=eror msg="unable to evaluate ngrok agent binary path for symlinks" obj=tunnels.session err="CreateFile C:\\Users\\Fayroz\\AppData\\Local\\ngrok\\ngrok.exe: The system cannot find the file specified."


🚀 Public URL: NgrokTunnel: "https://8cf8-196-137-30-105.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [19616]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
ERROR:pyngrok.process.ngrok:t=2025-05-25T10:03:24+0300 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=7325d62c462a clientid=ba3b9fad7266a80914b3d22a7cfd706f
ERROR:pyngrok.process.ngrok:t=2025-05-25T10:03:24+0300 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=14484bb7e18f err="session closed"
ERROR:pyngrok.process.ngrok:t=2025-05-25T10:03:34+0300 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="failed to dial ngrok server with address \"connect.ngrok-agent.com:443\": dial tcp 3.123.83.158:443: i/o timeout"
